In [20]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from utils import LSTM_basic, QA_Dataset, LSTM_onebyone
import sys
import pickle
import numpy as np
import json

torch.manual_seed(11)
corpus_path = './corpus.pkl'
embedding_path = './pretrained_embed.npy'
save_model = False
embedding_dim = 300
hidden_dim = 64
batch_size = 32
learning_rate = 1e-3
num_epochs = 100

def load_corpus(corpus_path):
    with open(corpus_path, 'rb') as handle:
        corpus = pickle.load(handle)
    return corpus

def load_embeddings(embed_path):
    return np.load(embed_path)

def save_checkpoint(state, fname):
    print("Save model to %s"%fname)
    torch.save(state, fname)

if __name__ == '__main__':
    corpus = load_corpus(corpus_path)
    embeddings = load_embeddings(embedding_path)
    # word2id = {y:x for x, y in corpus.items()}
    use_cuda = torch.cuda.is_available()
    # use_cuda = False
    model = LSTM_onebyone(embedding_dim = embedding_dim, hidden_dim = hidden_dim, vocab_size = len(corpus),\
                       batch_size = batch_size, use_cuda = use_cuda, embeddings = embeddings)
    print(model)
    if use_cuda:
        model = model.cuda()
    data_dir = './ready_data_onebyone'
    ftrain = 'mc500.train.tsv'
    fdev = 'mc500.dev.tsv'

    train_set = QA_Dataset(data_dir, ftrain, corpus, onebyone = True)
    dev_set = QA_Dataset(data_dir, fdev, corpus, onebyone = True)

    train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = 4)
    dev_loader = DataLoader(dev_set, batch_size = batch_size, shuffle = True, num_workers = 4)
    
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = learning_rate, weight_decay=1e-2)
    loss_function = nn.CrossEntropyLoss()
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []

    rawQuestions, rawAnswers, rawLabels, rawOutput = [], [], [], []

    # Training 
    for epoch in range(num_epochs):
        total_acc = 0.0
        total_loss = 0.0
        total = 0
        for iter, data in enumerate(train_loader):
            questions, answers, labels, _, _, _ = data
            labels = torch.squeeze(labels)
            l = labels
            if use_cuda:
                q, l = Variable(questions.cuda()).t(), labels.cuda()
                a = Variable(answers[0].cuda()).t()
                b = Variable(answers[1].cuda()).t()
                c = Variable(answers[2].cuda()).t()
                d = Variable(answers[3].cuda()).t()
            else:
                q = Variable(questions).t()
                a = Variable(answers[0]).t()
                b = Variable(answers[1]).t()
                c = Variable(answers[2]).t()
                d = Variable(answers[3]).t()
            model.batch_size = len(labels)
            output = model(q, a, b, c, d)
            loss = loss_function(output.cpu(), Variable(l).cpu())
            loss.backward()
            optimizer.step()

            # Calculating training accuracy
            _, pred = torch.max(output.data, 1)
            total_acc += (pred.cpu() == labels).sum()
            total += len(labels)
            total_loss += loss.data[0]
        train_loss_.append(total_loss / total)
        train_acc_.append(total_acc / total)
        if save_model:
            fname = '{}/Epoch_{}.model'.format('./checkpoints', epoch)
            save_checkpoint({'epoch': epoch + 1,
                             'state_dict': model.state_dict(),
                             'optimizer': optimizer.state_dict()},
                           fname = fname)

        # Testing/Validating
        total_acc = 0.0
        total_loss = 0.0
        total = 0
        for iter, data in enumerate(dev_loader):
            questions, answers, labels, origQuestions, origAnswers, origLabels = data
            labels = torch.squeeze(labels)
            l = labels
            if use_cuda:
                q, l = Variable(questions.cuda()).t(), labels.cuda()
                a = Variable(answers[0].cuda()).t()
                b = Variable(answers[1].cuda()).t()
                c = Variable(answers[2].cuda()).t()
                d = Variable(answers[3].cuda()).t()
            else:
                q = Variable(questions).t()
                a = Variable(answers[0]).t()
                b = Variable(answers[1]).t()
                c = Variable(answers[2]).t()
                d = Variable(answers[3]).t()

            model.batch_size = len(labels)
            output = model(q, a, b, c, d)
            loss = loss_function(output.cpu(), Variable(labels))
            if epoch == num_epochs - 1:
                rawQuestions.append(origQuestions)
                rawAnswers.append(origAnswers)
                rawLabels.append(origLabels)
                rawOutput.append(output.data.cpu().numpy())

            # Calculating training accuracy
            _, pred = torch.max(output.data, 1)
            total_acc += (pred.cpu() == labels).sum()
            total += len(labels)
            total_loss += loss.data[0]
        test_loss_.append(total_loss / total)
        test_acc_.append(total_acc / total)
        
        print("[Epoch %3d/%3d] Training loss: %.3f, Testing loss: %.3f, Training acc: %.3f, Testing acc: %.3f"%\
              (epoch, num_epochs, train_loss_[epoch], test_loss_[epoch],
                                 train_acc_[epoch], test_acc_[epoch]))

    correct, wrong = [], []
    for i in range(len(rawLabels)):
        for j in range(len(rawOutput[i])):
            # print(rawLabels[i][j], rawOutput[i][j])
            choice = int(np.argmax(rawOutput[i][j]))
            if rawLabels[i][j] == choice:
                correct.append((rawQuestions[i][j], rawAnswers[i][j], rawLabels[i][j], choice))
            else:
                wrong.append((rawQuestions[i][j], rawAnswers[i][j], rawLabels[i][j], choice))
    
    with open("correct_500.json", "w") as f:
        json.dump(correct, f)

    with open("wrong_500.json", "w") as f:
        json.dump(wrong, f)
                


LSTM_onebyone(
  (word_embeddings): Embedding(5666, 300)
  (lstm): LSTM(300, 64)
  (hidden2label): Linear(in_features=128, out_features=1)
  (softmax): Softmax()
)


C:\Users\WANG\Downloads\nn4nlp\nn4nlp\utils.py:161: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(torch.cat([qa, qb, qc, qd], 1))


[Epoch   0/100] Training loss: 0.044, Testing loss: 0.049, Training acc: 0.233, Testing acc: 0.255
[Epoch   1/100] Training loss: 0.044, Testing loss: 0.049, Training acc: 0.263, Testing acc: 0.255


KeyboardInterrupt: 

In [9]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from utils import LSTM_basic, QA_Dataset, LSTM_onebyone
import sys
import pickle
import numpy as np
import json

torch.manual_seed(11)
corpus_path = './corpus.pkl'
embedding_path = './pretrained_embed.npy'
save_model = False
embedding_dim = 300
hidden_dim = 64
batch_size = 16
learning_rate = 1e-3
num_epochs = 100

def load_corpus(corpus_path):
    with open(corpus_path, 'rb') as handle:
        corpus = pickle.load(handle)
    return corpus

def load_embeddings(embed_path):
    return np.load(embed_path)

def save_checkpoint(state, fname):
    print("Save model to %s"%fname)
    torch.save(state, fname)

if __name__ == '__main__':
    corpus = load_corpus(corpus_path)
    embeddings = load_embeddings(embedding_path)
    # word2id = {y:x for x, y in corpus.items()}
    use_cuda = torch.cuda.is_available()
    # use_cuda = False
    model = LSTM_onebyone(embedding_dim = embedding_dim, hidden_dim = hidden_dim, vocab_size = len(corpus),\
                       batch_size = batch_size, use_cuda = use_cuda, embeddings = embeddings)
    print(model)
    if use_cuda:
        model = model.cuda()
    data_dir = './ready_data_onebyone'
    ftrain = 'mc500.train.tsv'
    fdev = 'mc500.dev.tsv'

    train_set = QA_Dataset(data_dir, ftrain, corpus, onebyone = True)
    dev_set = QA_Dataset(data_dir, fdev, corpus, onebyone = True)

    train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = 4)
    dev_loader = DataLoader(dev_set, batch_size = batch_size, shuffle = True, num_workers = 4)
    
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = learning_rate, weight_decay=1e-2)
    loss_function = nn.CrossEntropyLoss()
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []

    rawQuestions, rawAnswers, rawLabels, rawOutput = [], [], [], []

    # Training 
    for epoch in range(num_epochs):
        total_acc = 0.0
        total_loss = 0.0
        total = 0
        for iter, data in enumerate(train_loader):
            questions, answers, labels, _, _, _ = data
            labels = torch.squeeze(labels)
            l = labels
            if use_cuda:
                q, l = Variable(questions.cuda()).t(), labels.cuda()
                a = Variable(answers[0].cuda()).t()
                b = Variable(answers[1].cuda()).t()
                c = Variable(answers[2].cuda()).t()
                d = Variable(answers[3].cuda()).t()
            else:
                q = Variable(questions).t()
                a = Variable(answers[0]).t()
                b = Variable(answers[1]).t()
                c = Variable(answers[2]).t()
                d = Variable(answers[3]).t()
            model.batch_size = len(labels)
            output = model(q, a, b, c, d)
            loss = loss_function(output.cpu(), Variable(l).cpu())
            loss.backward()
            optimizer.step()

            # Calculating training accuracy
            _, pred = torch.max(output.data, 1)
            total_acc += (pred.cpu() == labels).sum()
            total += len(labels)
            total_loss += loss.data[0]
        train_loss_.append(total_loss / total)
        train_acc_.append(total_acc / total)
        if save_model:
            fname = '{}/Epoch_{}.model'.format('./checkpoints', epoch)
            save_checkpoint({'epoch': epoch + 1,
                             'state_dict': model.state_dict(),
                             'optimizer': optimizer.state_dict()},
                           fname = fname)

        # Testing/Validating
        total_acc = 0.0
        total_loss = 0.0
        total = 0
        for iter, data in enumerate(dev_loader):
            questions, answers, labels, origQuestions, origAnswers, origLabels = data
            labels = torch.squeeze(labels)
            l = labels
            if use_cuda:
                q, l = Variable(questions.cuda()).t(), labels.cuda()
                a = Variable(answers[0].cuda()).t()
                b = Variable(answers[1].cuda()).t()
                c = Variable(answers[2].cuda()).t()
                d = Variable(answers[3].cuda()).t()
            else:
                q = Variable(questions).t()
                a = Variable(answers[0]).t()
                b = Variable(answers[1]).t()
                c = Variable(answers[2]).t()
                d = Variable(answers[3]).t()

            model.batch_size = len(labels)
            output = model(q, a, b, c, d)
            loss = loss_function(output.cpu(), Variable(labels))
            if epoch == num_epochs - 1:
                rawQuestions.append(origQuestions)
                rawAnswers.append(origAnswers)
                rawLabels.append(origLabels)
                rawOutput.append(output.data.cpu().numpy())

            # Calculating training accuracy
            _, pred = torch.max(output.data, 1)
            total_acc += (pred.cpu() == labels).sum()
            total += len(labels)
            total_loss += loss.data[0]
        test_loss_.append(total_loss / total)
        test_acc_.append(total_acc / total)
        
        print("[Epoch %3d/%3d] Training loss: %.3f, Testing loss: %.3f, Training acc: %.3f, Testing acc: %.3f"%\
              (epoch, num_epochs, train_loss_[epoch], test_loss_[epoch],
                                 train_acc_[epoch], test_acc_[epoch]))

    correct, wrong = [], []
    for i in range(len(rawLabels)):
        for j in range(len(rawOutput[i])):
            # print(rawLabels[i][j], rawOutput[i][j])
            choice = int(np.argmax(rawOutput[i][j]))
            if rawLabels[i][j] == choice:
                correct.append((rawQuestions[i][j], rawAnswers[i][j], rawLabels[i][j], choice))
            else:
                wrong.append((rawQuestions[i][j], rawAnswers[i][j], rawLabels[i][j], choice))
    
    with open("correct_500.json", "w") as f:
        json.dump(correct, f)

    with open("wrong_500.json", "w") as f:
        json.dump(wrong, f)
                


LSTM_onebyone(
  (word_embeddings): Embedding(5666, 300)
  (lstm): LSTM(300, 64)
  (hidden2label): Linear(in_features=128, out_features=1)
  (softmax): Softmax()
)


C:\Users\WANG\Downloads\nn4nlp\nn4nlp\utils.py:161: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(torch.cat([qa, qb, qc, qd], 1))


[Epoch   0/100] Training loss: 0.087, Testing loss: 0.090, Training acc: 0.226, Testing acc: 0.275
[Epoch   1/100] Training loss: 0.086, Testing loss: 0.090, Training acc: 0.267, Testing acc: 0.255
[Epoch   2/100] Training loss: 0.086, Testing loss: 0.091, Training acc: 0.266, Testing acc: 0.270
[Epoch   3/100] Training loss: 0.087, Testing loss: 0.090, Training acc: 0.247, Testing acc: 0.265
[Epoch   4/100] Training loss: 0.086, Testing loss: 0.090, Training acc: 0.257, Testing acc: 0.260
[Epoch   5/100] Training loss: 0.086, Testing loss: 0.091, Training acc: 0.274, Testing acc: 0.270
[Epoch   6/100] Training loss: 0.085, Testing loss: 0.091, Training acc: 0.292, Testing acc: 0.275
[Epoch   7/100] Training loss: 0.084, Testing loss: 0.091, Training acc: 0.310, Testing acc: 0.270
[Epoch   8/100] Training loss: 0.083, Testing loss: 0.091, Training acc: 0.320, Testing acc: 0.260
[Epoch   9/100] Training loss: 0.083, Testing loss: 0.091, Training acc: 0.327, Testing acc: 0.275
[Epoch  10

KeyboardInterrupt: 

In [21]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from utils import LSTM_basic, QA_Dataset, LSTM_onebyone
import sys
import pickle
import numpy as np
import json

torch.manual_seed(11)
corpus_path = './corpus.pkl'
embedding_path = './pretrained_embed.npy'
save_model = False
embedding_dim = 300
hidden_dim = 256
batch_size = 16
learning_rate = 1e-4
num_epochs = 100

def load_corpus(corpus_path):
    with open(corpus_path, 'rb') as handle:
        corpus = pickle.load(handle)
    return corpus

def load_embeddings(embed_path):
    return np.load(embed_path)

def save_checkpoint(state, fname):
    print("Save model to %s"%fname)
    torch.save(state, fname)

if __name__ == '__main__':
    corpus = load_corpus(corpus_path)
    embeddings = load_embeddings(embedding_path)
    # word2id = {y:x for x, y in corpus.items()}
    use_cuda = torch.cuda.is_available()
    # use_cuda = False
    model = LSTM_onebyone(embedding_dim = embedding_dim, hidden_dim = hidden_dim, vocab_size = len(corpus),\
                       batch_size = batch_size, use_cuda = use_cuda, embeddings = embeddings)
    print(model)
    if use_cuda:
        model = model.cuda()
    data_dir = './ready_data_onebyone'
    ftrain = 'mc500.train.tsv'
    fdev = 'mc500.dev.tsv'

    train_set = QA_Dataset(data_dir, ftrain, corpus, onebyone = True)
    dev_set = QA_Dataset(data_dir, fdev, corpus, onebyone = True)

    train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = 4)
    dev_loader = DataLoader(dev_set, batch_size = batch_size, shuffle = True, num_workers = 4)
    
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = learning_rate, weight_decay=1e-2)
    loss_function = nn.CrossEntropyLoss()
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []

    rawQuestions, rawAnswers, rawLabels, rawOutput = [], [], [], []

    # Training 
    for epoch in range(num_epochs):
        total_acc = 0.0
        total_loss = 0.0
        total = 0
        for iter, data in enumerate(train_loader):
            questions, answers, labels, _, _, _ = data
            labels = torch.squeeze(labels)
            l = labels
            if use_cuda:
                q, l = Variable(questions.cuda()).t(), labels.cuda()
                a = Variable(answers[0].cuda()).t()
                b = Variable(answers[1].cuda()).t()
                c = Variable(answers[2].cuda()).t()
                d = Variable(answers[3].cuda()).t()
            else:
                q = Variable(questions).t()
                a = Variable(answers[0]).t()
                b = Variable(answers[1]).t()
                c = Variable(answers[2]).t()
                d = Variable(answers[3]).t()
            model.batch_size = len(labels)
            output = model(q, a, b, c, d)
            loss = loss_function(output.cpu(), Variable(l).cpu())
            loss.backward()
            optimizer.step()

            # Calculating training accuracy
            _, pred = torch.max(output.data, 1)
            total_acc += (pred.cpu() == labels).sum()
            total += len(labels)
            total_loss += loss.data[0]
        train_loss_.append(total_loss / total)
        train_acc_.append(total_acc / total)
        if save_model:
            fname = '{}/Epoch_{}.model'.format('./checkpoints', epoch)
            save_checkpoint({'epoch': epoch + 1,
                             'state_dict': model.state_dict(),
                             'optimizer': optimizer.state_dict()},
                           fname = fname)

        # Testing/Validating
        total_acc = 0.0
        total_loss = 0.0
        total = 0
        for iter, data in enumerate(dev_loader):
            questions, answers, labels, origQuestions, origAnswers, origLabels = data
            labels = torch.squeeze(labels)
            l = labels
            if use_cuda:
                q, l = Variable(questions.cuda()).t(), labels.cuda()
                a = Variable(answers[0].cuda()).t()
                b = Variable(answers[1].cuda()).t()
                c = Variable(answers[2].cuda()).t()
                d = Variable(answers[3].cuda()).t()
            else:
                q = Variable(questions).t()
                a = Variable(answers[0]).t()
                b = Variable(answers[1]).t()
                c = Variable(answers[2]).t()
                d = Variable(answers[3]).t()

            model.batch_size = len(labels)
            output = model(q, a, b, c, d)
            loss = loss_function(output.cpu(), Variable(labels))
            if epoch == num_epochs - 1:
                rawQuestions.append(origQuestions)
                rawAnswers.append(origAnswers)
                rawLabels.append(origLabels)
                rawOutput.append(output.data.cpu().numpy())

            # Calculating training accuracy
            _, pred = torch.max(output.data, 1)
            total_acc += (pred.cpu() == labels).sum()
            total += len(labels)
            total_loss += loss.data[0]
        test_loss_.append(total_loss / total)
        test_acc_.append(total_acc / total)
        
        print("[Epoch %3d/%3d] Training loss: %.3f, Testing loss: %.3f, Training acc: %.3f, Testing acc: %.3f"%\
              (epoch, num_epochs, train_loss_[epoch], test_loss_[epoch],
                                 train_acc_[epoch], test_acc_[epoch]))

    correct, wrong = [], []
    for i in range(len(rawLabels)):
        for j in range(len(rawOutput[i])):
            # print(rawLabels[i][j], rawOutput[i][j])
            choice = int(np.argmax(rawOutput[i][j]))
            if rawLabels[i][j] == choice:
                correct.append((rawQuestions[i][j], rawAnswers[i][j], rawLabels[i][j], choice))
            else:
                wrong.append((rawQuestions[i][j], rawAnswers[i][j], rawLabels[i][j], choice))
    
    with open("correct_500.json", "w") as f:
        json.dump(correct, f)

    with open("wrong_500.json", "w") as f:
        json.dump(wrong, f)              

LSTM_onebyone(
  (word_embeddings): Embedding(5666, 300)
  (lstm): LSTM(300, 256)
  (hidden2label): Linear(in_features=512, out_features=1)
  (softmax): Softmax()
)


C:\Users\WANG\Downloads\nn4nlp\nn4nlp\utils.py:161: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(torch.cat([qa, qb, qc, qd], 1))


[Epoch   0/100] Training loss: 0.087, Testing loss: 0.090, Training acc: 0.233, Testing acc: 0.265
[Epoch   1/100] Training loss: 0.087, Testing loss: 0.090, Training acc: 0.273, Testing acc: 0.265
[Epoch   2/100] Training loss: 0.087, Testing loss: 0.090, Training acc: 0.275, Testing acc: 0.255
[Epoch   3/100] Training loss: 0.086, Testing loss: 0.090, Training acc: 0.283, Testing acc: 0.245
[Epoch   4/100] Training loss: 0.086, Testing loss: 0.090, Training acc: 0.296, Testing acc: 0.270
[Epoch   5/100] Training loss: 0.085, Testing loss: 0.090, Training acc: 0.292, Testing acc: 0.245
[Epoch   6/100] Training loss: 0.084, Testing loss: 0.090, Training acc: 0.308, Testing acc: 0.300
[Epoch   7/100] Training loss: 0.084, Testing loss: 0.091, Training acc: 0.316, Testing acc: 0.285
[Epoch   8/100] Training loss: 0.084, Testing loss: 0.090, Training acc: 0.329, Testing acc: 0.285
[Epoch   9/100] Training loss: 0.083, Testing loss: 0.091, Training acc: 0.315, Testing acc: 0.270
[Epoch  10

KeyboardInterrupt: 

In [16]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from utils import LSTM_basic, QA_Dataset, LSTM_onebyone
import sys
import pickle
import numpy as np
import json

torch.manual_seed(11)
corpus_path = './corpus.pkl'
embedding_path = './pretrained_embed.npy'
save_model = False
embedding_dim = 300
hidden_dim = 64
batch_size = 16
learning_rate = 1e-4
num_epochs = 20

def load_corpus(corpus_path):
    with open(corpus_path, 'rb') as handle:
        corpus = pickle.load(handle)
    return corpus

def load_embeddings(embed_path):
    return np.load(embed_path)

def save_checkpoint(state, fname):
    print("Save model to %s"%fname)
    torch.save(state, fname)

if __name__ == '__main__':
    corpus = load_corpus(corpus_path)
    embeddings = load_embeddings(embedding_path)
    # word2id = {y:x for x, y in corpus.items()}
    use_cuda = torch.cuda.is_available()
    # use_cuda = False
    model = LSTM_onebyone(embedding_dim = embedding_dim, hidden_dim = hidden_dim, vocab_size = len(corpus),\
                       batch_size = batch_size, use_cuda = use_cuda, embeddings = embeddings)
    print(model)
    if use_cuda:
        model = model.cuda()
    data_dir = './ready_data_onebyone'
    ftrain = 'mc160.train.tsv'
    fdev = 'mc160.dev.tsv'

    train_set = QA_Dataset(data_dir, ftrain, corpus, onebyone = True)
    dev_set = QA_Dataset(data_dir, fdev, corpus, onebyone = True)

    train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = 4)
    dev_loader = DataLoader(dev_set, batch_size = batch_size, shuffle = True, num_workers = 4)
    
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = learning_rate, weight_decay=1e-2)
    loss_function = nn.CrossEntropyLoss()
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []

    rawQuestions, rawAnswers, rawLabels, rawOutput = [], [], [], []

    # Training 
    for epoch in range(num_epochs):
        total_acc = 0.0
        total_loss = 0.0
        total = 0
        for iter, data in enumerate(train_loader):
            questions, answers, labels, _, _, _ = data
            labels = torch.squeeze(labels)
            l = labels
            if use_cuda:
                q, l = Variable(questions.cuda()).t(), labels.cuda()
                a = Variable(answers[0].cuda()).t()
                b = Variable(answers[1].cuda()).t()
                c = Variable(answers[2].cuda()).t()
                d = Variable(answers[3].cuda()).t()
            else:
                q = Variable(questions).t()
                a = Variable(answers[0]).t()
                b = Variable(answers[1]).t()
                c = Variable(answers[2]).t()
                d = Variable(answers[3]).t()
            model.batch_size = len(labels)
            output = model(q, a, b, c, d)
            loss = loss_function(output.cpu(), Variable(l).cpu())
            loss.backward()
            optimizer.step()

            # Calculating training accuracy
            _, pred = torch.max(output.data, 1)
            total_acc += (pred.cpu() == labels).sum()
            total += len(labels)
            total_loss += loss.data[0]
        train_loss_.append(total_loss / total)
        train_acc_.append(total_acc / total)
        if save_model:
            fname = '{}/Epoch_{}.model'.format('./checkpoints', epoch)
            save_checkpoint({'epoch': epoch + 1,
                             'state_dict': model.state_dict(),
                             'optimizer': optimizer.state_dict()},
                           fname = fname)

        # Testing/Validating
        total_acc = 0.0
        total_loss = 0.0
        total = 0
        for iter, data in enumerate(dev_loader):
            questions, answers, labels, origQuestions, origAnswers, origLabels = data
            labels = torch.squeeze(labels)
            l = labels
            if use_cuda:
                q, l = Variable(questions.cuda()).t(), labels.cuda()
                a = Variable(answers[0].cuda()).t()
                b = Variable(answers[1].cuda()).t()
                c = Variable(answers[2].cuda()).t()
                d = Variable(answers[3].cuda()).t()
            else:
                q = Variable(questions).t()
                a = Variable(answers[0]).t()
                b = Variable(answers[1]).t()
                c = Variable(answers[2]).t()
                d = Variable(answers[3]).t()

            model.batch_size = len(labels)
            output = model(q, a, b, c, d)
            loss = loss_function(output.cpu(), Variable(labels))
            if epoch == num_epochs - 1:
                rawQuestions.append(origQuestions)
                rawAnswers.append(origAnswers)
                rawLabels.append(origLabels)
                rawOutput.append(output.data.cpu().numpy())

            # Calculating training accuracy
            _, pred = torch.max(output.data, 1)
            total_acc += (pred.cpu() == labels).sum()
            total += len(labels)
            total_loss += loss.data[0]
        test_loss_.append(total_loss / total)
        test_acc_.append(total_acc / total)
        
        print("[Epoch %3d/%3d] Training loss: %.3f, Testing loss: %.3f, Training acc: %.3f, Testing acc: %.3f"%\
              (epoch, num_epochs, train_loss_[epoch], test_loss_[epoch],
                                 train_acc_[epoch], test_acc_[epoch]))

    correct, wrong = [], []
    for i in range(len(rawLabels)):
        for j in range(len(rawOutput[i])):
            # print(rawLabels[i][j], rawOutput[i][j])
            choice = int(np.argmax(rawOutput[i][j]))
            if rawLabels[i][j] == choice:
                correct.append((rawQuestions[i][j], rawAnswers[i][j], rawLabels[i][j], choice))
            else:
                wrong.append((rawQuestions[i][j], rawAnswers[i][j], rawLabels[i][j], choice))
    
    with open("correct_160.json", "w") as f:
        json.dump(correct, f)

    with open("wrong_160.json", "w") as f:
        json.dump(wrong, f)
                


LSTM_onebyone(
  (word_embeddings): Embedding(5666, 300)
  (lstm): LSTM(300, 64)
  (hidden2label): Linear(in_features=128, out_features=1)
  (softmax): Softmax()
)


C:\Users\WANG\Downloads\nn4nlp\nn4nlp\utils.py:161: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(torch.cat([qa, qb, qc, qd], 1))


[Epoch   0/ 20] Training loss: 0.089, Testing loss: 0.092, Training acc: 0.211, Testing acc: 0.208
[Epoch   1/ 20] Training loss: 0.089, Testing loss: 0.092, Training acc: 0.221, Testing acc: 0.217
[Epoch   2/ 20] Training loss: 0.089, Testing loss: 0.092, Training acc: 0.229, Testing acc: 0.208
[Epoch   3/ 20] Training loss: 0.089, Testing loss: 0.092, Training acc: 0.264, Testing acc: 0.208
[Epoch   4/ 20] Training loss: 0.089, Testing loss: 0.092, Training acc: 0.271, Testing acc: 0.225
[Epoch   5/ 20] Training loss: 0.089, Testing loss: 0.092, Training acc: 0.293, Testing acc: 0.225
[Epoch   6/ 20] Training loss: 0.089, Testing loss: 0.092, Training acc: 0.293, Testing acc: 0.225
[Epoch   7/ 20] Training loss: 0.089, Testing loss: 0.092, Training acc: 0.304, Testing acc: 0.225
[Epoch   8/ 20] Training loss: 0.089, Testing loss: 0.092, Training acc: 0.307, Testing acc: 0.233
[Epoch   9/ 20] Training loss: 0.089, Testing loss: 0.092, Training acc: 0.307, Testing acc: 0.233
[Epoch  10

In [18]:
fdev = 'mc160.test.tsv'
dev_set = QA_Dataset(data_dir, fdev, corpus, onebyone = True)

dev_loader = DataLoader(dev_set, batch_size = batch_size, shuffle = True, num_workers = 4)
rawQuestions, rawAnswers, rawLabels, rawOutput = [], [], [], []
total_acc = 0.0
total_loss = 0.0
total = 0
for iter, data in enumerate(dev_loader):
    questions, answers, labels, origQuestions, origAnswers, origLabels = data
    labels = torch.squeeze(labels)
    l = labels
    if use_cuda:
        q, l = Variable(questions.cuda()).t(), labels.cuda()
        a = Variable(answers[0].cuda()).t()
        b = Variable(answers[1].cuda()).t()
        c = Variable(answers[2].cuda()).t()
        d = Variable(answers[3].cuda()).t()
    else:
        q = Variable(questions).t()
        a = Variable(answers[0]).t()
        b = Variable(answers[1]).t()
        c = Variable(answers[2]).t()
        d = Variable(answers[3]).t()

    model.batch_size = len(labels)
    output = model(q, a, b, c, d)
    loss = loss_function(output.cpu(), Variable(labels))
    if epoch == num_epochs - 1:
        rawQuestions.append(origQuestions)
        rawAnswers.append(origAnswers)
        rawLabels.append(origLabels)
        rawOutput.append(output.data.cpu().numpy())

correct, wrong = [], []
for i in range(len(rawLabels)):
    for j in range(len(rawOutput[i])):
        # print(rawLabels[i][j], rawOutput[i][j])
        choice = int(np.argmax(rawOutput[i][j]))
        if rawLabels[i][j] == choice:
            correct.append((rawQuestions[i][j], rawAnswers[i][j], rawLabels[i][j], choice))
        else:
            wrong.append((rawQuestions[i][j], rawAnswers[i][j], rawLabels[i][j], choice))
    
with open("correct_160.json", "w") as f:
    json.dump(correct, f)

with open("wrong_160.json", "w") as f:
    json.dump(wrong, f)

C:\Users\WANG\Downloads\nn4nlp\nn4nlp\utils.py:161: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(torch.cat([qa, qb, qc, qd], 1))
